## Install

In [1]:
!pip3 install huggingface-hub

In [2]:
!pip install --upgrade openai
!pip install --upgrade pydantic

  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/30/1d/27c3571504fb6fb1e9f7c906d93590ead22f5f34910489e155ee28512eeb/openai-1.3.5-py3-none-any.whl.metadata
  Obtaining dependency information for httpx<1,>=0.23.0 from https://files.pythonhosted.org/packages/a2/65/6940eeb21dcb2953778a6895281c179efd9100463ff08cb6232bb6480da7/httpx-0.25.2-py3-none-any.whl.metadata
  Obtaining dependency information for httpcore==1.* from https://files.pythonhosted.org/packages/56/ba/78b0a99c4da0ff8b0f59defa2f13ca4668189b134bd9840b6202a93d9a0f/httpcore-1.0.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.8/220.8 kB 5.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.0 MB/s eta 0:00:00
  Obtaining dependency information for pydantic from https://files.pythonhosted.org/packages/0a/2b/64066de1c4cf3d4ed623beeb3bbf3f8d0cc26661f1e7d1

In [3]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U peft
%pip install -U accelerate
%pip install -U trl

In [ ]:
# !mkdir MetaMath-Mistral-7B
# !huggingface-cli download meta-math/MetaMath-Mistral-7B --local-dir MetaMath-Mistral-7B --local-dir-use-symlinks False

## Import library

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, wandb
from datasets import load_dataset
from trl import SFTTrainer

/opt/conda/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [5]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_hf = user_secrets.get_secret("HUGGINGFACE_TOKEN")
secret_wandb = user_secrets.get_secret("wandb")

In [6]:
!huggingface-cli login --token $secret_hf

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
# wandb.login(key = secret_wandb)
# run = wandb.init(
# #     project='Fine tuning MetaMath mistral 7B - ZAIC',
#     project='Fine tuning openchat 7B - ZAIC',
#     job_type="training", 
#     anonymous="allow"
# )

In [8]:
# base_model = "meta-math/MetaMath-Mistral-7B"
base_model = "openchat/openchat_3.5"
new_model = "BK-BigAI-Math"
model_hotamath_path = "/kaggle/working/BK-BigAI-Math"

## Download dataset

In [9]:
!mkdir dataset

In [10]:
from huggingface_hub import hf_hub_download
hf_hub_download(repo_id="hotamago/ZAIC-2023", filename="Elementary Maths Solving/test.zip", revision="main", repo_type="dataset", local_dir="dataset", local_dir_use_symlinks=False)
hf_hub_download(repo_id="hotamago/ZAIC-2023", filename="Elementary Maths Solving/train.zip", revision="main", repo_type="dataset", local_dir="dataset", local_dir_use_symlinks=False)

'dataset/Elementary Maths Solving/train.zip'

In [11]:
!sudo apt-get install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-25ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [12]:
!mkdir datasetRaw
!unzip -q -o "dataset/Elementary Maths Solving/test.zip" -d "datasetRaw"
!unzip -q -o "dataset/Elementary Maths Solving/train.zip" -d "datasetRaw"

## Load dataset

In [13]:
import os
import json
import re
import time

In [14]:
train_data = None
test_data = None
with open(os.path.join("datasetRaw", "train", "/kaggle/working/datasetRaw/math_train.json"), "r") as f:
    train_data = json.loads(f.read())['data']
with open(os.path.join("datasetRaw", "test", "/kaggle/working/datasetRaw/math_test.json"), "r") as f:
    test_data = json.loads(f.read())['data']

In [15]:
train_data[0]

{'id': '1',
 'question': 'Một người bán hàng bỏ ra 80,000 đồng tiền vốn và bị lỗ 6%. Để tính số tiền lỗ ta phải tính',
 'choices': ['A. 80,000 : 6',
  'B. 80,000 x 6',
  'C. 80,000 : (6 x 100)',
  'D. (80,000 x 6) : 100'],
 'explanation': 'Theo đề bài, số tiền lỗ bằng 6% của 80 000 đồng . Để tìm số tiền lỗ ta có thể lấy 80 000 chia cho 100 rồi nhân với 6 (tức là 80 000 : 100 × 6) hoặc lấy 80000 nhân với 6 rồi chia cho 100 (tức là 80 000 × 6 : 100).',
 'answer': 'D. (80,000 x 6) : 100'}

In [16]:
test_data[0]

{'id': '01-0203',
 'question': 'Một cửa hàng đã bán 30% số hàng hiện có và thu được 15 000 000 đồng. Hỏi nếu bán hết hàng thì cửa hàng thu được bao nhiêu tiền?',
 'choices': ['A. 4 500 000 đồng',
  'B. 45 000 000 đồng',
  'C. 50 000 000 đồng',
  'D. 450 000 000 đồng']}

In [17]:
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "<|end_of_turn|>" # "</s>"
DEFAULT_BOS_TOKEN = "<s>"
DEFAULT_UNK_TOKEN = "<unk>"
DEFAULT_BOI_TOKEN = "Human:" # "[INST]"
DEFAULT_EOI_TOKEN = "Assistant:" # "[/INST]"
PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with the choices, one of the choices is the correct answer to the request. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Choices:\n{choices}"
    ),
    "prompt_input_run": (
        DEFAULT_BOI_TOKEN + " Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Choices:\n{choices}"
        "\n" + DEFAULT_EOI_TOKEN + " \n\n"
        "### Explanation:\n Let's think step by step.\n"
    ),
}

In [18]:
timeGlobal = 0
def startTime():
    global timeGlobal
    timeGlobal = time.time()
def getTime():
    return (time.time() - timeGlobal)

In [19]:
def ApplyPromptTemplate(instruction, choices, typeP = "prompt_input"):
    return PROMPT_DICT[typeP].format(instruction = instruction, choices = "\n".join(choices))

### Add explantion by GPT 3.5 Tubo

In [20]:
from openai import OpenAI
client = OpenAI(
    api_key=user_secrets.get_secret("OPENAI_API_KEY"),
    organization='org-j48waUrvSOM1n0J1SLXiAr8n',
)

def autoGPTAddExplantion(problem, answer):
    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": "Explan by Vietnamese step-by-step for given answer to given problem.\nRule:\n- No markdown format\n- Given answer always true for given problem\n- No title\n- Short explantion\n"},
        {"role": "user", "content": f"### Problem:\n{problem}\n\n### Answer:\n{answer}"},
        {"role": "system", "content": "Giải thích: "},
      ],
#       max_tokens=512,
      temperature=0,
      top_p=1.0,
#       top_k=50,
    )
    return response.choices[0].message.content

In [21]:
import time

In [22]:
startTime()
cntSt = 0
i = 0
timeWait = 20
while i < len(train_data):
    singleData = train_data[i]
    if "explanation" in singleData.keys():
        i += 1
        continue
    print(f"Runing testcase ({i})")
    
    try:
        res = autoGPTAddExplantion(singleData['question'], singleData['answer'])
    except Exception as inst:
        print(type(inst))
        print(inst)
        
        time.sleep(timeWait)
        timeWait *= 2
        if timeWait > 80:
            break
        print
        continue
    
    timeWait = 20
        
    cntSt += 1
    train_data[i]['explanation'] = res
    print(f"Done testcase ({i})")
    #     print(train_data[i])
    if cntSt%3 == 0:
        deltaTime = getTime()
        if (60 - deltaTime) > -1:
            time.sleep(60 - deltaTime + 2)
        startTime()
        cntSt = 0
    
    i += 1

Runing testcase (512)
Done testcase (512)
Runing testcase (514)
Done testcase (514)
Runing testcase (518)
Done testcase (518)
Runing testcase (520)
Done testcase (520)
Runing testcase (522)
Done testcase (522)
Runing testcase (523)
Done testcase (523)
Runing testcase (524)
Done testcase (524)
Runing testcase (525)
Done testcase (525)
Runing testcase (527)
Done testcase (527)
Runing testcase (529)
Done testcase (529)
Runing testcase (530)
Done testcase (530)
Runing testcase (531)
Done testcase (531)
Runing testcase (532)
Done testcase (532)
Runing testcase (533)
Done testcase (533)
Runing testcase (535)
Done testcase (535)
Runing testcase (536)
Done testcase (536)
Runing testcase (537)
Done testcase (537)
Runing testcase (538)
Done testcase (538)
Runing testcase (539)
Done testcase (539)
Runing testcase (540)
Done testcase (540)
Runing testcase (541)
Done testcase (541)
Runing testcase (542)
Done testcase (542)
Runing testcase (543)
Done testcase (543)
Runing testcase (545)
Done testcas

In [24]:
with open("/kaggle/working/datasetRaw/math_train.json", "w", encoding='utf-8') as f:
    json.dump({
        "__count__": len(train_data),
        "data": train_data
    }, f)

### Add data by education program

Todo: Create data generation algorithms for each chapter.

Below is class base, change \_call function to a function that create radom question for that chapter

In [ ]:
class BaseGenChapter:
    def __init__(*args, **kwargs):
        pass
    
    def _call(self, *args, **kwargs) -> str:
        raise NotImplementedError("_call not Implement yet")
    
    def call(self, *args, **kwargs) -> str:
        return self._call(*args, **kwargs)

### Preprocess data

In [ ]:
from datasets import Dataset

In [ ]:
num_train_dataset = len(train_data)
valition_radio = 0.05
tokenized_train_dataset_raw = train_data[:-int(num_train_dataset*valition_radio)]
tokenized_val_dataset_raw = train_data[-int(num_train_dataset*valition_radio):]

In [ ]:
datasetStruct = {"input":[], "output":[]}
dataset = {"text":[]}
num_train_dataset = len(tokenized_train_dataset_raw)
for i in range(num_train_dataset):
    ttdro = tokenized_train_dataset_raw[i]
    
    if "explanation" not in ttdro.keys():
        continue
    
    input_content = "{0} {1}".format(
        DEFAULT_BOI_TOKEN,
        ApplyPromptTemplate(ttdro['question'], ttdro['choices']),
    )
    datasetStruct["input"].append(input_content)
    
    if "explanation" not in ttdro.keys():
        output_content = "\n{0} \n\n{1}\n\n{2} {3}".format(
            DEFAULT_EOI_TOKEN,
            "### Explanation:\nNo explanation{0}",
            "### Answer:\n{0}".format(ttdro['answer']),
            DEFAULT_EOS_TOKEN,
        )
    else:
        output_content = "\n{0} \n\n{1}\n\n{2} {3}".format(
            DEFAULT_EOI_TOKEN,
            "### Explanation:\n{0}".format(ttdro['explanation']),
            "### Answer:\n{0}".format(ttdro['answer']),
            DEFAULT_EOS_TOKEN,
        )
    datasetStruct["output"].append(output_content)
    
    dataset["text"].append(input_content + output_content)
    
dataset = Dataset.from_dict(dataset)
#     <s>[INST][/INST] </s>

In [ ]:
print(dataset["text"][32])

In [ ]:
print(len(dataset["text"]))

## Train model

### Load model

In [ ]:
bnb_config = BitsAndBytesConfig(  
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
#         model_hotamath_path,
    load_in_4bit=True,
#     load_in_8bit= True,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
#         torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model,
    model_max_length=512,
    padding_side="right",
    use_fast=False,
)

In [ ]:
special_tokens_dict = {
    'additional_special_tokens': [DEFAULT_BOI_TOKEN, DEFAULT_EOI_TOKEN],
    'pad_token': DEFAULT_PAD_TOKEN,
    'bos_token': DEFAULT_BOS_TOKEN,
    'eos_token': DEFAULT_EOS_TOKEN,
    'unk_token': DEFAULT_UNK_TOKEN,
}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

In [ ]:
print(tokenizer.encode("{0} Hello, how are you? \n{1} I'm fine, thank you!{2}".format(
    DEFAULT_BOI_TOKEN,
    DEFAULT_EOI_TOKEN,
    DEFAULT_EOS_TOKEN,
)))

### DataCollator

In [ ]:
from trl import DataCollatorForCompletionOnlyLM

In [ ]:
instruction_template = DEFAULT_BOI_TOKEN
response_template = DEFAULT_EOI_TOKEN
collator = DataCollatorForCompletionOnlyLM(instruction_template=instruction_template, response_template=response_template, tokenizer=tokenizer, mlm=False)

### Setup model for train

In [ ]:
print(model)

In [ ]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)

In [ ]:
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=100,
    logging_steps=10,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    save_total_limit = 3,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="wandb"
)

In [ ]:
model = model.to("cuda:0")

In [ ]:
print(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
print_trainable_parameters(model)

In [ ]:
trainer = SFTTrainer(
    model=model,
#     model = base_model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=512,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
    data_collator=collator,
)

In [ ]:
model.config.use_cache = False

### Train and save

In [ ]:
trainer.train()

In [ ]:
# model.save_pretrained(new_model)
trainer.model.save_pretrained(new_model)
model.config.to_json_file(os.path.join(new_model, "config.json"))
tokenizer.save_pretrained(new_model)
wandb.finish()
model.config.use_cache = True

In [ ]:
!zip -r BK-BigAI-Math.zip BK-BigAI-Math

In [ ]:
from IPython.display import FileLink
FileLink(r'BK-BigAI-Math.zip')

## Evalution

### Download model

In [ ]:
from huggingface_hub import hf_hub_download
hf_hub_download(repo_id="hotamago/ZAIC-2023-Model", filename="Hota-Math.zip", repo_type="model", local_dir="/kaggle/working/", local_dir_use_symlinks=False)

In [ ]:
!unzip -q -o Hota-Math.zip -d ./

In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

### Load model

In [ ]:
model_name_or_path = "/kaggle/working/BK-BigAI-Math"
# bnb_config = BitsAndBytesConfig(  
#     load_in_4bit= True,
#     bnb_4bit_quant_type= "nf4",
#     bnb_4bit_compute_dtype= torch.bfloat16,
#     bnb_4bit_use_double_quant= False,
# )
# model = AutoModelForCausalLM.from_pretrained(
#         model_name_or_path,
#         load_in_4bit=True,
#         quantization_config=bnb_config,
#         torch_dtype=torch.bfloat16,
#         device_map="auto",
#         trust_remote_code=True,
# )

# model = AutoModelForCausalLM.from_pretrained(
#         model_name_or_path,
#         torch_dtype=torch.bfloat16,
#         device_map="auto",
#         trust_remote_code=True,
# )

model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
    load_in_4bit=True,
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
    model_max_length=512,
    padding_side="right",
    use_fast=False,
)

In [ ]:
# special_tokens_dict = {'additional_special_tokens': ['[INST]','[/INST]']}
# num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

### Setup pipeline with auto answer get

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.01,
    top_p=1,
    top_k=50,
    repetition_penalty=1.1,
    pad_token_id=tokenizer.eos_token_id
)

In [ ]:
import random
# globalRegxCompire = "0-9a-zA-Z\.\:\-\^\! "
def niceValueToCompire(x):
#     x = re.sub("[^{0}]".format(globalRegxCompire), "", x)
    x = re.sub("[ \t\n]", "", x)
    return x
def autoLLMFormat(question, choises = None, debug=False):
    prompt_template = ApplyPromptTemplate(question, choises, "prompt_input_run")
    res = pipe(prompt_template)[0]['generated_text']
    if debug:
        print(res)
    x = re.findall("### Answer:[\n ](.+)", res)
    
    if choises == None:
        return x
    
    choises_compare = [niceValueToCompire(choise_pred) for choise_pred in choises]

    if len(x) == 0:
        return choises[random.randrange(0, len(choises))]
    
    x = niceValueToCompire(x[0])
    
    if (x not in choises_compare):
        return choises[random.randrange(0, len(choises))]
    
    for i in range(len(choises_compare)):
        if x == choises_compare[i]:
            return choises[i]
    
    return "wtf"

### Run evalution

In [ ]:
count_proc_testcase = 0
count_pass_testcase = 0

In [ ]:
while count_proc_testcase < len(tokenized_val_dataset_raw):
    tvdo = tokenized_val_dataset_raw[count_proc_testcase]
    startTime()
    answer = autoLLMFormat(tvdo['question'], tvdo['choices'], True)
    deltaTime = getTime()

    if answer == tvdo['answer']:
        count_pass_testcase += 1
    
    count_proc_testcase += 1
    print("Testcase {0}, time: {1}, answer: {2} | {3}, Passed: {4}, IsPass: {5}".format(
        count_proc_testcase,
        deltaTime,
        answer,
        tvdo['answer'],
        count_pass_testcase,
        (answer == tvdo['answer'])
    ))

## Run public test dataset

In [ ]:
result_test = []
if os.path.exists(os.path.join("result", "result.txt")):
    with open(os.path.join("result", "result.txt"), "r") as f:
        result_test = f.read().split("\n")
count_id = len(result_test)

In [ ]:
!mkdir result

In [ ]:
print(len(test_data))

In [ ]:
while count_id < len(test_data):
    one_test_data = test_data[count_id]
    startTime()
    answer = autoLLMFormat(one_test_data['question'], one_test_data['choices'], False)
    deltaTime = getTime()
    result_test.append("{0}".format(answer))
    count_id += 1
    if count_id%10 == 0:
        with open(os.path.join("result", "result.txt"), "w", encoding='utf-8') as f:
            f.write("\n".join(result_test))
    print("Testcase {0}, time: {1}, answer: {2}".format(count_id, deltaTime, answer))

In [ ]:
with open(os.path.join("result", "result.txt"), "w", encoding='utf-8') as f:
    f.write("\n".join(result_test))

In [ ]:
print("\n".join(result_test))